# MO444A - PROJETO 3

Alunas:
*   Maria Tejada Begazo (RA 197488)
*   Nayara Ferreira da Silva (RA 223804)

# PART I - CODIGO

In [57]:
import os
import random
import numpy as np

os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0

%cd search

[Errno 2] No such file or directory: 'search'
/home/mtejada/Desktop/PacamanML/197488_/search


## ALGORITMO GENETICO


Nós modelamos nosso algoritmo genetico esta modelado em que temos 4 opções básica determinadas por um if-then-else.
Onde nossos terminaies estão determinados por:

* d1: distancia entre o fantasma mas cercano. 
* d2: distancia entre la pildora mas cercana. 
* a1: escapar de un fantasma
* a2: ir a comer una pildora cercana
* a3: movimiento aleatorio. 

Ademáis vimos la opção de só ter como recompensa o score médio de 3 jogos seguidos. 
Nosso individuos estão formado por 8 parametros, por exemplo:

    [op, d1, d2, dist1, dist2, a1, a2, a3]

onde op se refiere à opção que pode ser de 1 à 4, depois d1 e d2 que se informo na parte de em cima, dist1 e dist2 podem estar no rango de 1 à 5 e a1, a2, a3 que se refierem às ações. 

Nosso crossver em dois pontos:

    parent1 =[1, d1, d2, 3, 4, a1, a2, a3]    => child1 =[1, d2, d1, 1, 2, a1, a2, a3] 
    
    parent2 =[3, d2, d1, 1, 2, a3, a2, a1]    => child2 =[3, d1, d2, 3, 4, a3, a2, a1]
    
A mutação esta determinado nos pontos que são enteiros, onde trocara o número de forma aleatoria. 


In [58]:
def initialization_of_population(size):
    op = [1, 2, 3, 4]
    d  = ['d1', 'd2']
    dis= [1, 2, 3, 4]
    a  = ['a1', 'a2', 'a3']
    
    popul = []
    for i in range(size):
        indiv = list(np.ones(len(op) + len(d) + len(dis) + len(a)))
        random.shuffle(op)
        random.shuffle(d)
        random.shuffle(dis)
        random.shuffle(a)
        indiv[0] = op[0]
        indiv[1:3] = d
        indiv[3] = dis[0]
        random.shuffle(dis)
        indiv[4] = dis[0]
        indiv[5:] = a
        popul.append(indiv.copy())
    return popul
        

In [59]:
def crossover_twopoints(parent1, parent2, PROBABILITY_CROSSOVER):
    child1 = parent1.copy()
    child2 = parent2.copy()
    probability = np.random.uniform(0,1)
    
    if probability < PROBABILITY_CROSSOVER:
        for i in range(1,5):
            child1[i] = parent2[i]
            child2[i] = parent1[i]
    return child1, child2

def mutation_point(indiv, PROBABILITY_MUTATION):
    index_mut = [0, 3, 4]
    random.shuffle(index_mut)
    probability = np.random.uniform(0,1)
    
    if probability < PROBABILITY_MUTATION:
        indiv[index_mut[0]] = random.randint(1,4)
    return indiv


In [60]:
def selection_tournament(population, scores):
    indx1 = random.randint(0, len(population)-1)
    indx2 = random.randint(0, len(population)-1)
    
    if scores[indx1] < scores[indx2]:
        return population[indx2]
    else:
        return population[indx1]

In [61]:
def algorithmGenetic(Environment, size, iteration, PROBABILITY_CROSSOVER, PROBABILITY_MUTATION):
    pop = initialization_of_population(size)
    
    nameFile = "GAindividual.txt"
    nameFileGeneration = "GAGeneration.txt"
    nameFileData = "GAData.txt"
    score_aver = []
    score_best = []
    score_pior = []
    
    dataFile = open(nameFileData, 'w')
    generationFile = open(nameFileGeneration, 'w')
    dataFile.write("Avr_better,Avr,Avr_pior\n")
    
    for i in range(iteration):
        score = []
        best_score = -99999
        best_Indx_indiv = 0
        
        for j in range(size):
            generationFile.write(str(i) + ",")
            for k in range(len(pop[j])):
                generationFile.write(str(pop[j][k]) + ",")
                
            with open(nameFile, 'w') as f:
                f.write(str(pop[j]))
                f.write("\n")
            
            p = ['layout:  smallClassic', 'Pacman died! Score: -482', 'Pacman died! Score: -412', 'Pacman died! Score: -410', 'Average Score: -434.6666666666667']
            if Environment == 0:
                p = !python3 pacman.py -n 3 -l smallClassic -p GAagents --agentArgs "tableMode=Small,f=GAindividual.txt"
            elif Environment == 1:
                p = !python3 pacman.py -n 3 -l mediumClassic -p GAagents --agentArgs "tableMode=Medium,f=GAindividual.txt"
            else:
                p = !python3 pacman.py -n 3 -l originalClassic -p GAagents --agentArgs "tableMode=Original,f=GAindividual.txt"
            
            print(p)
            score.append(int(float(p[4].split(' ')[2])))
            
            if(score[j]  > best_score):
                best_score = score[j]
                best_Indx_indiv = j
            
            generationFile.write(str(score[j]))
            generationFile.write('\n')
            generationFile.flush()
        
        q = pop.copy()
        indx = 0
        for j in range(int(size/2)):
            mom = selection_tournament(pop, score)
            dad = selection_tournament(pop, score)
            child1, child2 = crossover_twopoints(mom, dad, PROBABILITY_CROSSOVER)
            q[indx] = mutation_point(child1, PROBABILITY_MUTATION)
            q[indx+1] = child2
            indx+=2
        
        score_best.append(max(score))
        score_aver.append(sum(score)/size)
        score_pior.append(min(score))
        
        dataFile.write(str(max(score)) + "," + str(sum(score)/size) + "," + str(min(score)) + "\n")
        dataFile.flush()
        
        print(score_best, score_aver, score_pior)
        q[0] = pop[best_Indx_indiv]
        pop = q
    
    return pop[0],score_aver, score_best, score_pior


In [ ]:
bestInd, score_aver, score_best, score_pior = algorithmGenetic(1, 25,50, 0.70, 0.20 )

['layout:  mediumClassic', 'Pacman died! Score: -389', 'Pacman died! Score: -491', 'Pacman died! Score: -381', 'Average Score: -420.3333333333333', 'Scores:        -389.0, -491.0, -381.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -433', 'Pacman died! Score: -451', 'Pacman died! Score: -488', 'Average Score: -457.3333333333333', 'Scores:        -433.0, -451.0, -488.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -469', 'Pacman died! Score: -465', 'Pacman died! Score: -466', 'Average Score: -466.6666666666667', 'Scores:        -469.0, -465.0, -466.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -445', 'Pacman died! Score: -450', 'Pacman died! Score: -468', 'Average Score: -454.3333333333333', 'Scores:        -445.0, -450.0, -468.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, L

['layout:  mediumClassic', 'Pacman died! Score: -436', 'Pacman died! Score: -504', 'Pacman died! Score: -314', 'Average Score: -418.0', 'Scores:        -436.0, -504.0, -314.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -306', 'Pacman died! Score: -486', 'Pacman died! Score: -383', 'Average Score: -391.6666666666667', 'Scores:        -306.0, -486.0, -383.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -159', 'Pacman died! Score: -316', 'Pacman died! Score: -283', 'Average Score: -252.66666666666666', 'Scores:        -159.0, -316.0, -283.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -477', 'Pacman died! Score: -517', 'Pacman died! Score: -466', 'Average Score: -486.6666666666667', 'Scores:        -477.0, -517.0, -466.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']

['layout:  mediumClassic', 'Pacman died! Score: -319', 'Pacman died! Score: -412', 'Pacman died! Score: -199', 'Average Score: -310.0', 'Scores:        -319.0, -412.0, -199.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -177', 'Pacman died! Score: -227', 'Pacman died! Score: -247', 'Average Score: -217.0', 'Scores:        -177.0, -227.0, -247.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -577', 'Pacman died! Score: -101', 'Pacman died! Score: -302', 'Average Score: -326.6666666666667', 'Scores:        -577.0, -101.0, -302.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -459', 'Pacman died! Score: -464', 'Pacman died! Score: -375', 'Average Score: -432.6666666666667', 'Scores:        -459.0, -464.0, -375.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  m

['layout:  mediumClassic', 'Pacman died! Score: -504', 'Pacman died! Score: -380', 'Pacman died! Score: -388', 'Average Score: -424.0', 'Scores:        -504.0, -380.0, -388.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
[-199, -148, -155, -65] [-404.08, -382.28, -361.8, -302.2] [-596, -570, -533, -530]
['layout:  mediumClassic', 'Pacman died! Score: -69', 'Pacman died! Score: -206', 'Pacman died! Score: -145', 'Average Score: -140.0', 'Scores:        -69.0, -206.0, -145.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: 13', 'Pacman died! Score: -333', 'Pacman died! Score: -275', 'Average Score: -198.33333333333334', 'Scores:        13.0, -333.0, -275.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -366', 'Pacman died! Score: -206', 'Pacman died! Score: -284', 'Average Score: -285.3333333333333', 'Scores:        -366.0, -206.0, -284.0

['layout:  mediumClassic', 'Pacman died! Score: -287', 'Pacman died! Score: -387', 'Pacman died! Score: -310', 'Average Score: -328.0', 'Scores:        -287.0, -387.0, -310.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -382', 'Pacman died! Score: -157', 'Pacman died! Score: -358', 'Average Score: -299.0', 'Scores:        -382.0, -157.0, -358.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -306', 'Pacman died! Score: 63', 'Pacman died! Score: -414', 'Average Score: -219.0', 'Scores:        -306.0, 63.0, -414.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -167', 'Pacman died! Score: -402', 'Pacman died! Score: -447', 'Average Score: -338.6666666666667', 'Scores:        -167.0, -402.0, -447.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', '

...

['layout:  mediumClassic', 'Pacman died! Score: -221', 'Pacman died! Score: -373', 'Pacman died! Score: -402', 'Average Score: -332.0', 'Scores:        -221.0, -373.0, -402.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -283', 'Pacman died! Score: -100', 'Pacman died! Score: -178', 'Average Score: -187.0', 'Scores:        -283.0, -100.0, -178.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -47', 'Pacman died! Score: -268', 'Pacman died! Score: -384', 'Average Score: -233.0', 'Scores:        -47.0, -268.0, -384.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -103', 'Pacman died! Score: -414', 'Pacman died! Score: -195', 'Average Score: -237.33333333333334', 'Scores:        -103.0, -414.0, -195.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic'

['layout:  mediumClassic', 'Pacman died! Score: -237', 'Pacman died! Score: -196', 'Pacman died! Score: -189', 'Average Score: -207.33333333333334', 'Scores:        -237.0, -196.0, -189.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -389', 'Pacman died! Score: -369', 'Pacman died! Score: 95', 'Average Score: -221.0', 'Scores:        -389.0, -369.0, 95.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -401', 'Pacman died! Score: -320', 'Pacman died! Score: -162', 'Average Score: -294.3333333333333', 'Scores:        -401.0, -320.0, -162.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -303', 'Pacman died! Score: -364', 'Pacman died! Score: -180', 'Average Score: -282.3333333333333', 'Scores:        -303.0, -364.0, -180.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['l

['layout:  mediumClassic', 'Pacman died! Score: -83', 'Pacman died! Score: -184', 'Pacman died! Score: -289', 'Average Score: -185.33333333333334', 'Scores:        -83.0, -184.0, -289.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -25', 'Pacman died! Score: -118', 'Pacman died! Score: -193', 'Average Score: -112.0', 'Scores:        -25.0, -118.0, -193.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -2', 'Pacman died! Score: -98', 'Pacman died! Score: -352', 'Average Score: -150.66666666666666', 'Scores:        -2.0, -98.0, -352.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -298', 'Pacman died! Score: -391', 'Pacman died! Score: -316', 'Average Score: -335.0', 'Scores:        -298.0, -391.0, -316.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumCla

['layout:  mediumClassic', 'Pacman died! Score: -269', 'Pacman died! Score: -382', 'Pacman died! Score: -307', 'Average Score: -319.3333333333333', 'Scores:        -269.0, -382.0, -307.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -160', 'Pacman died! Score: 11', 'Pacman died! Score: 80', 'Average Score: -23.0', 'Scores:        -160.0, 11.0, 80.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -308', 'Pacman died! Score: -317', 'Pacman died! Score: -449', 'Average Score: -358.0', 'Scores:        -308.0, -317.0, -449.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -488', 'Pacman died! Score: -524', 'Pacman died! Score: -457', 'Average Score: -489.6666666666667', 'Scores:        -488.0, -524.0, -457.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
[-199, -148, -155, -6

['layout:  mediumClassic', 'Pacman died! Score: 183', 'Pacman died! Score: -163', 'Pacman died! Score: -357', 'Average Score: -112.33333333333333', 'Scores:        183.0, -163.0, -357.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -358', 'Pacman died! Score: -244', 'Pacman died! Score: -401', 'Average Score: -334.3333333333333', 'Scores:        -358.0, -244.0, -401.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -414', 'Pacman died! Score: -227', 'Pacman died! Score: -378', 'Average Score: -339.6666666666667', 'Scores:        -414.0, -227.0, -378.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['layout:  mediumClassic', 'Pacman died! Score: -244', 'Pacman died! Score: -204', 'Pacman died! Score: -77', 'Average Score: -175.0', 'Scores:        -244.0, -204.0, -77.0', 'Win Rate:      0/3 (0.00)', 'Record:        Loss, Loss, Loss']
['l

## REINFORCEMENT LEARNING


## RESULTADOS

In [48]:
nameFile = "GAindividual.txt"

with open(nameFile, 'w') as f:
    f.write(str(bestInd))
    f.write("\n")

In [105]:
p = !python3 pacman.py -n 1 -l smallClassic -p GAagents --agentArgs "tableMode=Small,f=GAindividual.txt"
p

['layout:  smallClassic',
 "[0, 'd2', 'd1', 2, 2, 'a3', 'a2', 'a1']",
 'Pacman died! Score: -468',
 'Average Score: -468.0',
 'Scores:        -468.0',
 'Win Rate:      0/1 (0.00)',
 'Record:        Loss']